In [9]:
"""
Simple CNN Object Detection Example and Performance Evaluation Metrics Discussion

This script demonstrates a simplified CNN-based object detection pipeline using TensorFlow/Keras.
It includes:
- A simple CNN model for object classification (as a proxy for detection)
- Dummy dataset generation for demonstration
- Training and evaluation
- Calculation of common object detection performance metrics

Note: Real object detection models like YOLO, SSD, Faster R-CNN are complex and require specialized architectures and datasets.
This example is for educational purposes only.

"""

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Generate dummy dataset
def generate_dummy_data(num_samples=1000, img_size=(28, 28), num_classes=2):
    # Generate random images
    X = np.random.rand(num_samples, img_size[0], img_size[1], 1).astype(np.float32)
    # Generate random labels (0 or 1)
    y = np.random.randint(0, num_classes, size=(num_samples,))
    return X, y

# Define a simple CNN model for classification
def create_cnn_model(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Calculate Intersection over Union (IoU) for bounding boxes
def iou(boxA, boxB):
    # box = [x1, y1, x2, y2]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# Dummy function to simulate object detection predictions and ground truth
def generate_dummy_bboxes(num_samples=100):
    # Each sample has one ground truth box and one predicted box
    gt_boxes = []
    pred_boxes = []
    pred_scores = []
    for _ in range(num_samples):
        # Ground truth box: random coordinates
        gt = [np.random.randint(0, 20), np.random.randint(0, 20),
              np.random.randint(21, 40), np.random.randint(21, 40)]
        # Predicted box: random coordinates with some noise
        pred = [gt[0] + np.random.randint(-5, 5), gt[1] + np.random.randint(-5, 5),
                gt[2] + np.random.randint(-5, 5), gt[3] + np.random.randint(-5, 5)]
        # Clamp values to positive
        pred = [max(0, p) for p in pred]
        gt_boxes.append(gt)
        pred_boxes.append(pred)
        # Random confidence score
        pred_scores.append(np.random.rand())
    return gt_boxes, pred_boxes, pred_scores

# Evaluate detection performance using IoU threshold
def evaluate_detection(gt_boxes, pred_boxes, pred_scores, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    for i in range(len(gt_boxes)):
        iou_score = iou(gt_boxes[i], pred_boxes[i])
        if iou_score >= iou_threshold and pred_scores[i] >= 0.5:
            tp += 1
        elif pred_scores[i] >= 0.5:
            fp += 1
        else:
            fn += 1
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

def main():
    # Generate dummy data
    X_train, y_train = generate_dummy_data(num_samples=800)
    X_test, y_test = generate_dummy_data(num_samples=200)

    # Create model
    model = create_cnn_model(input_shape=(28,28,1), num_classes=2)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train model
    model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=2)

    # Evaluate model on test data
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print(f"Test accuracy: {test_acc:.4f}")

    # Predict on test data
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Calculate classification metrics
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred, average='binary')
    print(f"Classification Precision: {precision:.4f}")
    print(f"Classification Recall: {recall:.4f}")
    print(f"Classification F1 Score: {f1:.4f}")

    # Simulate object detection bounding boxes and scores
    gt_boxes, pred_boxes, pred_scores = generate_dummy_bboxes(num_samples=200)

    # Evaluate detection performance
    det_precision, det_recall, det_f1 = evaluate_detection(gt_boxes, pred_boxes, pred_scores)
    print(f"Detection Precision: {det_precision:.4f}")
    print(f"Detection Recall: {det_recall:.4f}")
    print(f"Detection F1 Score: {det_f1:.4f}")

    # Discussion of performance evaluation metrics for object detection:
    print("\nPerformance Evaluation Metrics for Object Detection:")
    print("1. Intersection over Union (IoU): Measures the overlap between predicted and ground truth bounding boxes.")
    print("2. Precision: The ratio of true positive detections to all positive detections (TP / (TP + FP)).")
    print("3. Recall: The ratio of true positive detections to all ground truth objects (TP / (TP + FN)).")
    print("4. F1 Score: Harmonic mean of precision and recall, balancing both metrics.")
    print("5. Mean Average Precision (mAP): Average precision across all classes and IoU thresholds, commonly used in object detection challenges.")
    print("6. Confusion Matrix: Shows true positives, false positives, false negatives, and true negatives for classification tasks.")
    print("7. Precision-Recall Curve: Plots precision vs recall at different confidence thresholds.")
    print("8. Average Recall (AR): Average recall over multiple IoU thresholds.")
    print("\nThese metrics help evaluate how well an object detection algorithm identifies and localizes objects in images.")

if __name__ == "__main__":
    main()


ImportError: Traceback (most recent call last):
  File "c:\Users\Sakshi\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorflow\python\__init__.py", line 66, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "c:\Users\Sakshi\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "c:\Users\Sakshi\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 15, in swig_import_helper
    import imp
ModuleNotFoundError: No module named 'imp'


Failed to load the native TensorFlow runtime.

See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/get_started/os_setup.md#import_error

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.